In [1]:
# Define the path to the multi-FASTA file
fasta_file = 'dna2.fasta'

## 1. Sequence records in the file
How many records are in the file? A record in a FASTA file is defined as a single-line header, followed by lines of sequence data. The header line is distinguished from the sequence data by a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier of the sequence, and the rest of the line is an optional description of the entry. There should be no space between the ">" and the first letter of the identifier. 

In [2]:
# Initialize a variable to store the number of records in file
records = 0

In [3]:
# Open the FASTA file in read mode using with statement
with open(fasta_file, 'r') as file:
    # Iterate through each line in the file
    for line in file:
        # Remove all trailing characters from the ends of each line
        line.strip()
        # Check for the occurrence of ">" at the beginning of each line
        if line.startswith(">"):
            # When an occurrence is found, increase the value of the records variable by 1
            records+=1
# Output the final value of the records variable, depicting the number of records in the given file
print(records)

18


## 2. Lengths of the sequences
What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence? Is there more than one longest or shortest sequence? What are their identifiers? 

In [4]:
# Initialize an empty list to store lengths of sequences
seq_lengths = []

In [5]:
# Initialize an empty dictionary to store the sequence IDs and their corresponding lengths
seq_len_dict = {}

In [6]:
# Import SeqIO module from Biopython package
from Bio import SeqIO

In [7]:
# Parse through the multi-FASTA file and store its contents as a list
sequences = list(SeqIO.parse(fasta_file, 'fasta'))

In [8]:
# Iterate through the list of FASTA records
for seq_record in sequences:
    seq_len_dict[seq_record.id] = len(seq_record.seq)
print(seq_len_dict)

{'gi|142022655|gb|EQ086233.1|91': 4635, 'gi|142022655|gb|EQ086233.1|304': 1151, 'gi|142022655|gb|EQ086233.1|255': 4894, 'gi|142022655|gb|EQ086233.1|45': 3511, 'gi|142022655|gb|EQ086233.1|396': 4076, 'gi|142022655|gb|EQ086233.1|250': 2867, 'gi|142022655|gb|EQ086233.1|322': 442, 'gi|142022655|gb|EQ086233.1|88': 890, 'gi|142022655|gb|EQ086233.1|594': 967, 'gi|142022655|gb|EQ086233.1|293': 4338, 'gi|142022655|gb|EQ086233.1|75': 1352, 'gi|142022655|gb|EQ086233.1|454': 4564, 'gi|142022655|gb|EQ086233.1|16': 4804, 'gi|142022655|gb|EQ086233.1|584': 964, 'gi|142022655|gb|EQ086233.1|4': 2095, 'gi|142022655|gb|EQ086233.1|277': 1432, 'gi|142022655|gb|EQ086233.1|346': 115, 'gi|142022655|gb|EQ086233.1|527': 2646}


In [9]:
# Iterate through all the values in the dictionary and append lengths to the 'seq_lengths' list
for values in seq_len_dict.values():
    seq_lengths.append(values)

In [10]:
# Initialize variables to store maximum and minimum length
max_len = max(seq_lengths)
min_len = min(seq_lengths)

# Pring the longest and shortest lengths
print(f"Maximum length: {max_len}")
print(f"Minimum length: {min_len}")

Maximum length: 4894
Minimum length: 115


## 3. ORFs
In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5'). For instance, the three possible forward reading frames for the sequence AGGTGACACCGCAAGCCTTATATTAGC are: 

AGG TGA CAC CGC AAG CCT TAT ATT AGC

A GGT GAC ACC GCA AGC CTT ATA TTA GC

AG GTG ACA CCG CAA GCC TTA TAT TAG C 

These are called reading frames 1, 2, and 3 respectively. An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.

Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to identify all ORFs present in each sequence of the FASTA file, and answer the following questions: what is the length of the longest ORF in the file? What is the identifier of the sequence containing the longest ORF? For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence. For instance, the following ORF in reading frame 1:

\>sequence1

ATGCCCTAG

starts at position 1.

Note that because the following sequence:

\>sequence2

ATGAAAAAA

does not have any stop codon in reading frame 1, we do not consider it to be an ORF in reading frame 1.

In [25]:
def find_orfs(sequence, frame):
    """Identifies ORFs in a given sequence for a specific reading frame."""
    start_codon = 'ATG'
    stop_codons = {'TAA', 'TGA', 'TAG'}
    
    orfs = []
    start_positions = []
    seq_len = len(sequence)
    
    for i in range(frame, seq_len, 3):
        codon = sequence[i:i+3]
        if codon == start_codon:
            start_positions.append(i)
        elif codon in stop_codons:
            if start_positions:
                for start in start_positions:
                    orf = sequence[start:i+3]
                    orfs.append((start, orf))
                start_positions = []  # Reset after finding a stop codon
    
    return orfs

def orf(file):
    """Identifies the ORFs in a given multi-fasta file for all three reading frames, 
    and returns a dictionary containing the sequence IDs and their corresponding ORFs in each frame."""
    
    from Bio import SeqIO
    sequences = list(SeqIO.parse(file, 'fasta'))
    
    orf_dict1 = {}
    orf_dict2 = {}
    orf_dict3 = {}
    
    for seq_record in sequences:
        seq_id = seq_record.id
        sequence = str(seq_record.seq)
        
        # Find ORFs in all three reading frames
        orfs1 = find_orfs(sequence, 0)
        orfs2 = find_orfs(sequence, 1)
        orfs3 = find_orfs(sequence, 2)
        
        # Store the ORFs in the respective dictionaries
        orf_dict1[seq_id] = orfs1
        orf_dict2[seq_id] = orfs2
        orf_dict3[seq_id] = orfs3
    
    return orf_dict1, orf_dict2, orf_dict3

In [26]:
# Usage example
orf_dict1, orf_dict2, orf_dict3 = orf(fasta_file)

In [27]:
# Print the ORFs for the first sequence in the first reading frame
for seq_id, orfs in orf_dict1.items():
    print(f"Sequence ID: {seq_id}")
    for start, orf in orfs:
        print(f"Start: {start}, ORF: {orf}")
    break  # Remove this line if you want to print all sequences

Sequence ID: gi|142022655|gb|EQ086233.1|91
Start: 228, ORF: ATGCCGGCTTTCGCGATCGGCGCGAACACGCCGGCCGGCCTGCTCGCGTGGGGCTTGCCGGCGAATGCGTCGGCGGGCGGTGCGCTCGACAACCGCGTGTGGGGCGTCCAGGTGAACAATGCGGTGAAGTACGTGAGCCCGACGTTCGGCGGATTGTCGTTCGGCGGCCTGTGGGGCTTCGGCAACGTGCCCGGCACGGTCGCGCGCAGCAGCGTGCAAAGCGCGATGCTGTCCTACACGCAAGGCGCGTTCAGCGCCGCGCTCGCTTATTTCGGCCAGCACGATGTAACTGCCGGTGGCAATCTGCGCAATTTCTCGGGCGGTGCAGGCTACAACGTCGGGCAGTTCCGCGTCTTCGGCATGGTGTCGGACGTGCGGATCAGCGCCGCCGCGCCGCTGCGGGCCACGACCTATGACGGCGGCTTGACCTATGCGGTCACGCCGGCGTTGCAGCTCGGCGGCGGCTTCCAGTACCAGCAGCGCGGCGGCGACATCGGCTCGGCCAACCAGGTCACGTTGAGCGCCGACTATTCGCTGTCGAAGCGTACCGGCCTTTACGTGGTATTCGCACGCGGGCACGACAGTGCGTATGGCGCGCAGGTCGAGGCGGCGCTCGGCGGGGCGGCGTCCGGCTCGACGCAGACCGCGGTCCGGCTCGGGCTGCGGCATCAGTTCTGA
Start: 453, ORF: ATGCTGTCCTACACGCAAGGCGCGTTCAGCGCCGCGCTCGCTTATTTCGGCCAGCACGATGTAACTGCCGGTGGCAATCTGCGCAATTTCTCGGGCGGTGCAGGCTACAACGTCGGGCAGTTCCGCGTCTTCGGCATGGTGTCGGACGTGCGGATCAGCGCCGCCGCGCCGCTGCGGGCCACGACCTATGACGGCGGCTTGACCTATGCGGTCACGCCGGCGTTGCAGCTCGGCGGCGGCTTCC

### (ii) Finding the length and sequence ID of the longest ORF in each reading frame

In [33]:
def find_longest_orf(orf_dict):
    """Finds the longest ORF in a given dictionary of ORFs and returns its length, sequence ID, and starting position."""
    longest_orf = None
    longest_length = 0
    longest_seq_id = None
    longest_start = None
    
    for seq_id, orfs in orf_dict.items():
        for start, orf in orfs:
            orf_length = len(orf)
            if orf_length > longest_length:
                longest_length = orf_length
                longest_orf = orf
                longest_seq_id = seq_id
                longest_start = start  # Capture the starting position of the longest ORF
    
    return longest_seq_id, longest_length, longest_orf, longest_start

In [34]:
# Find the longest ORF in each reading frame and their starting positions
longest_seq_id1, longest_length1, longest_orf1, longest_start1 = find_longest_orf(orf_dict1)
longest_seq_id2, longest_length2, longest_orf2, longest_start2 = find_longest_orf(orf_dict2)
longest_seq_id3, longest_length3, longest_orf3, longest_start3 = find_longest_orf(orf_dict3)

# Print the results including the starting positions
print(f"Frame 1 - Longest ORF: Sequence ID: {longest_seq_id1}, Length: {longest_length1}, Start: {longest_start1}")
print(f"Frame 2 - Longest ORF: Sequence ID: {longest_seq_id2}, Length: {longest_length2}, Start: {longest_start2}")
print(f"Frame 3 - Longest ORF: Sequence ID: {longest_seq_id3}, Length: {longest_length3}, Start: {longest_start3}")

Frame 1 - Longest ORF: Sequence ID: gi|142022655|gb|EQ086233.1|45, Length: 2394, Start: 384
Frame 2 - Longest ORF: Sequence ID: gi|142022655|gb|EQ086233.1|16, Length: 1458, Start: 3070
Frame 3 - Longest ORF: Sequence ID: gi|142022655|gb|EQ086233.1|527, Length: 1821, Start: 635


### (iii) Retrieving the sequence, length and starting position of the longest ORF corresponding to a sequence ID

In [31]:
def get_longest_orf_for_id(seq_id, orf_dict1, orf_dict2, orf_dict3):
    """Finds the longest ORF for a specific sequence ID across all three reading frames."""
    longest_orf = ""
    longest_length = 0
    longest_start = -1
    longest_frame = None

    def update_longest_orf(orfs, frame):
        nonlocal longest_orf, longest_length, longest_start, longest_frame
        for start, orf in orfs:
            orf_length = len(orf)
            if orf_length > longest_length:
                longest_length = orf_length
                longest_orf = orf
                longest_start = start
                longest_frame = frame

    if seq_id in orf_dict1:
        update_longest_orf(orf_dict1[seq_id], 1)
    if seq_id in orf_dict2:
        update_longest_orf(orf_dict2[seq_id], 2)
    if seq_id in orf_dict3:
        update_longest_orf(orf_dict3[seq_id], 3)

    return longest_orf, longest_length, longest_start, longest_frame

In [32]:
# Usage example
target_seq_id = 'gi|142022655|gb|EQ086233.1|16'

# Find the longest ORF for the target sequence ID
longest_orf, longest_length, longest_start, longest_frame = get_longest_orf_for_id(target_seq_id, orf_dict1, orf_dict2, orf_dict3)

# Output the result
print(f"Sequence ID: {target_seq_id}")
print(f"Longest ORF in frame {longest_frame}: Start = {longest_start}, Length = {longest_length}")
print(f"ORF Sequence: {longest_orf}")

Sequence ID: gi|142022655|gb|EQ086233.1|16
Longest ORF in frame 3: Start = 1439, Length = 1644
ORF Sequence: ATGCGGGCCATCCTGCATCGCCGCCTTTCGTTCCACCCGGGCCGGCATCGAGTGATGCCGGCGTTGACGTTTTCGTGGAGTGAGTCAGATGAATCACGCAGCGAATCCCGCCGATCCCGATCGCGCCGCGGCGCAGGGCGGCAGCCTGTACAACGACGATCTCGCGCCGACGACGCCGGCGCAGCGCACGTGGAAGTGGTATCACTTCGCGGCGCTGTGGGTCGGGATGGTGATGAACATCGCGTCGTACATGCTCGCGGCCGGGCTGATCCAGGAAGGCATGTCGCCGTGGCAGGCGGTGACGACGGTGCTGCTCGGCAACCTGATCGTGCTCGTGCCGATGCTGCTGATCGGCCATGCGGGCGCGAAGCACGGGATTCCGTACGCGGTGCTCGTGCGCGCGTCGTTCGGCACGCAGGGGGCGAAGCTGCCGGCGCTGCTGCGCGCGATCGTCGCGTGCGGCTGGTACGGGATCCAGACCTGGCTCGGCGGCAGCGCGATCTATACGCTGCTGAACATCCTGACCGGCAACGCGCTGCATGGCGCCGCGCTGCCGGTCATCGGCATCGGGTTCGGGCAGCTCGCATGCTTCCTCGTGTTCTGGGCGCTGCAGCTCTACTTCATCTGGCATGGCACCGATTCGATCCGCTGGCTCGAAAGCTGGTCGGCGCCGATCAAGGTCGTGATGTGCGTGGCGCTGGTGTGGTGGGCAACGTCGAAGGCGGGCGGCTTCGGCACGATGCTGTCGGCGCCGTCGCAGTTTGCCGCAGGCGGCAAGAAAGCCGGGCTGTTCTGGGCGACCTTCTGGCCGGGGCTGACCGCGATGGTCGGCTTCTGGGCGACGCTCGCGCTGAACATCCCCGACTTCACGCGCTTCGCGCATTCGCAGCG

## 4. Repeats
 A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.

In [19]:
def find_repeats(fasta_file, n):
    '''Identifies repeats of a given length 'n' from sequences in a given fasta file, and returns the repeats and their corresponding frequencies in a dictionary '''
    # Initialize empty lists to store sequences and the substrings
    fasta_sequences = []
    substrings = []
    
    # Import SeqIO from Biopython library
    from Bio import SeqIO
    
    # Parse through the multi-FASTA file and store its contents as a list
    sequences = list(SeqIO.parse(fasta_file, 'fasta'))
    
    # Iterate through the list of records and store sequences in a separate list
    for seq_record in sequences:
        fasta_sequences.append(seq_record.seq)
    
    # Loop through the list of sequences and extract the substrings of length 'n'
    for seq in fasta_sequences:
        for i in range(len(seq) - n + 1):
            substring = seq[i:i+n]
            substrings.append(substring)
    
    #Import Counter module from collections package
    from collections import Counter
    
    # Count the occurrences of each substring
    substring_counts = Counter(substrings)
    
    # Remove entries with a count of 1
    repeat_dict = {sub: count for sub, count in substring_counts.items() if count > 1}
    
    # Find the substring with the highest frequency
    max_substring = max(repeat_dict, key=repeat_dict.get)
    max_frequency = repeat_dict[max_substring]

    return repeat_dict, print(f"The substring with the highest frequency is '{max_substring}' with a count of {max_frequency}.")



In [36]:
find_repeats(fasta_file, 12)

The substring with the highest frequency is 'CATTCGCCATTC' with a count of 10.


({Seq('CGGCGTGTCGCG'): 2,
  Seq('TCGTCGGCGATC'): 2,
  Seq('CGGCGCGAACAC'): 2,
  Seq('GGCGCGAACACG'): 2,
  Seq('GCGCGAACACGC'): 2,
  Seq('CGCGAACACGCC'): 2,
  Seq('CGAACACGCCGG'): 2,
  Seq('GAACACGCCGGC'): 2,
  Seq('CACGCCGGCCGG'): 2,
  Seq('GGCGGCCTGTGG'): 2,
  Seq('GCGGCCTGTGGG'): 2,
  Seq('CGCGTTCAGCGC'): 3,
  Seq('GCGTTCAGCGCC'): 2,
  Seq('CGTTCAGCGCCG'): 2,
  Seq('TACAACGTCGGG'): 2,
  Seq('TGTCGGACGTGC'): 2,
  Seq('GCGGCGGCTTCC'): 2,
  Seq('CGGCTTCCAGTA'): 2,
  Seq('GGCTTCCAGTAC'): 2,
  Seq('GCTTCCAGTACC'): 2,
  Seq('AGCAGCGCGGCG'): 2,
  Seq('GCAGCGCGGCGG'): 2,
  Seq('CAGCGCGGCGGC'): 2,
  Seq('GCGGCGACATCG'): 2,
  Seq('CAACCAGGTCAC'): 2,
  Seq('AACCAGGTCACG'): 2,
  Seq('CACGCGGGCACG'): 2,
  Seq('GGCGGCGTCCGG'): 2,
  Seq('GCGGCGTCCGGC'): 3,
  Seq('TCTGACGATGCG'): 2,
  Seq('CTGACGATGCGC'): 2,
  Seq('TGCGGCCCGACG'): 2,
  Seq('GCCGAGCACGTG'): 2,
  Seq('CCGAGCACGTGC'): 2,
  Seq('CGAGCATCATCG'): 2,
  Seq('GAGCATCATCGA'): 2,
  Seq('AGCATCATCGAC'): 2,
  Seq('TTGGTCACGTCG'): 2,
  Seq('CTGCT